In [1]:
import numpy as np
import pandas as pd
from utility import utility as util
from utility import utils as utils
import yaml

In [2]:
import math
import os

In [3]:
import numpy.random as rand

In [4]:
from tqdm import tqdm

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from real_funcOnlineAlgo import create_ev_dict_from_df

In [6]:
import latest_deterministic_solver_ev_penetration
import generate_expected_EV_values as E_ev_generator

In [7]:
import ev_data_sampler

In [8]:
def create_ev_dict(seed):
    
    rng = np.random.default_rng(seed)

    FINAL_SOC = 0.97
    ALPHA_C = 11
    B_CAP = 80
    ETA_C = 0.98

    ev_dict = {}
    ev_outer_keys = ['init_soc','ev_stay_t','ev_laxity']

    for key in ev_outer_keys:
        ev_dict[key] = {}

    for key in ev_outer_keys:
        for h in range(24):
            ev_dict[key]['hour_{}'.format(str(h))] = []
                 
    for hour in range(24):
        num_arrived_ev = ev_data_sampler.sample_num_EV_arrivals(rng, hour)

        for _ in range(num_arrived_ev):
            stay_t = ev_data_sampler.sample_ev_stay_time(rng, hour)
            init_soc = ev_data_sampler.sample_init_soc(seed)
            seed += 1
            depart_time = hour + stay_t
            if(depart_time > 23):
                depart_time = 23
                stay_t = depart_time - hour
            laxity = stay_t - (FINAL_SOC - init_soc)*B_CAP/(ALPHA_C * ETA_C)
            if(laxity >= 0):
                ev_dict['ev_stay_t']['hour_{}'.format(hour)].append(stay_t)
                ev_dict['init_soc']['hour_{}'.format(hour)].append(init_soc)
                ev_dict['ev_laxity']['hour_{}'.format(hour)].append(laxity)
                
    return ev_dict

In [9]:
def create_forecast_ev_dict(seed, sample_no):
    
    rng = np.random.default_rng(seed)
    ev_rng = np.random.default_rng(seed+sample_no)

    
    DAY_HRS = 24
    FINAL_SOC = 0.97
    ALPHA_C = 11
    B_CAP = 80
    ETA_C = 0.98
    NOISE_STD = 0.1

    ev_dict = {}
    ev_outer_keys = ['init_soc','ev_stay_t','ev_laxity']

    for key in ev_outer_keys:
        ev_dict[key] = {}

    for key in ev_outer_keys:
        for h in range(DAY_HRS):
            ev_dict[key]['hour_{}'.format(str(h))] = []
                 
    for hour in range(DAY_HRS):
        num_arrived_ev = ev_data_sampler.sample_num_EV_arrivals(rng, hour)
        
#         print('in forecast ', num_arrived_ev)
        
        num_arrived_ev_noise = ev_rng.normal(0, (num_arrived_ev*NOISE_STD))
        
        num_arrived_ev += num_arrived_ev_noise
#         if(num_arrived_ev_noise > 0):
#             num_arrived_ev = np.ceil(num_arrived_ev)
        num_arrived_ev = int(num_arrived_ev)
        
        if(num_arrived_ev <= 0):
            num_arrived_ev = 0
        
        for _ in range(num_arrived_ev):
            stay_t = ev_data_sampler.sample_ev_stay_time(rng, hour)
            stay_t_noise = ev_rng.normal(0, (stay_t*NOISE_STD))
            stay_t += stay_t_noise
            if(stay_t_noise > 0):
                stay_t = np.ceil(stay_t)
            stay_t = int(stay_t)

            
            if(stay_t >= (DAY_HRS-1)):
                stay_t = DAY_HRS-1
            elif(stay_t < 0):
                stay_t = 0
            
            init_soc = ev_data_sampler.sample_init_soc(seed)
            init_soc_noise = ev_rng.normal(0, (init_soc*NOISE_STD))
            init_soc += init_soc_noise
            seed += 1

            depart_time = hour + stay_t
            if(depart_time > 23):
                depart_time = 23
                stay_t = depart_time - hour
                
            if(init_soc >= FINAL_SOC):
                init_soc = FINAL_SOC
            elif(init_soc < 0):
                init_soc = 0
                                
            laxity = stay_t - (((FINAL_SOC - init_soc)*B_CAP)/(ALPHA_C * ETA_C))
            if(laxity >= 0):
                ev_dict['ev_stay_t']['hour_{}'.format(hour)].append(stay_t)
                ev_dict['init_soc']['hour_{}'.format(hour)].append(init_soc)
                ev_dict['ev_laxity']['hour_{}'.format(hour)].append(laxity)
                
    return ev_dict

In [10]:
class EV:
    
    def __init__(self, arrival_time, stay_time, soc_init, laxity):
        self.arrival_time = arrival_time
        self.stay_time = stay_time
        self.departure_time = self.arrival_time + self.stay_time
        self.soc_final = 0.97
        self.soc_init = soc_init
        self.soc_t = soc_init
        self.laxity = laxity
        self.battery_cap = 80 #kW
        self.alpha_c = 11 #kW
        self.eta_c = 0.98
        self.alpha_d = -11 #kW
        self.eta_d = 0.98
        self.allow_discharge = False
        self.priority_charge = False
        self.bool_c_d = False
        self.completed = False
        self.time_to_full_soc = 0
        self.incentive_valuation = 0
        self.actual_payback = 0
        self.discharge_threshold = 1.25

In [11]:
def get_connected_ev(ev_dict, hour): 
    
    just_arrived_ev_lst = []

    if(len(ev_dict['init_soc']['hour_{}'.format(str(hour))]) != 0):
        for ev_num in range(len(ev_dict['init_soc']['hour_{}'.format(str(hour))])):
            just_arrived_ev_lst.append(EV(hour,
                                         ev_dict['ev_stay_t']['hour_{}'.format(str(hour))][ev_num],
                                         ev_dict['init_soc']['hour_{}'.format(str(hour))][ev_num],
                                          ev_dict['ev_laxity']['hour_{}'.format(str(hour))][ev_num]
                                         ))
        return just_arrived_ev_lst
    else:
        return [-1]

In [12]:
def surplus_pv_gen(current_time, pv_diff, available_ev_lst, im_price_lst):

    imbalance_sell = 0
    e_extra = pv_diff
    
    charge_e_dict = {}

    for ev in available_ev_lst:
        if(ev.bool_c_d == False and ev.soc_t < ev.soc_final):
            charge_e_dict[ev] = ev.laxity


    # sort based on ascending order of Laxity. Smaller lax => more urgent to charge!
    if(len(charge_e_dict) != 0):
        charge_e_dict = sorted(charge_e_dict.items(), key=lambda x: x[1], reverse=False)

        for ev in charge_e_dict:
            if(e_extra > 0):
                e_charging = charge_EV(ev[0], e_extra)
                e_extra -= e_charging
#                 if(im_price_lst[current_time] > 0):
#                     imbalance_sell += e_charging * im_price_lst[current_time]
    
    # Sell remaining surplus energy to imbalance market after EV charging complete!
    if(e_extra > 0 and im_price_lst[current_time] > 0):
        imbalance_sell += e_extra  * im_price_lst[current_time]
    return imbalance_sell

In [13]:
def charge_EV(ev, e_charging):
    
    ETA_C = 0.98
    
    e_required = (ev.soc_final - ev.soc_t) * ev.battery_cap
    
    if(e_charging >= e_required):
        e_charging = e_required
        e_required = 0

    if(e_charging >= ev.alpha_c):
        e_charging = ev.alpha_c
    
    ev.soc_t =  ev.soc_t + (e_charging*ETA_C)/ev.battery_cap
    ev.stay_time -= 1
    ev.laxity = ev.stay_time - ((ev.soc_final - ev.soc_t) * ev.battery_cap)/(ev.alpha_c * ETA_C)
    if(ev.laxity == 0):
        ev.laxity = 0
    ev.bool_c_d = True
    
    return e_charging

In [14]:
def shortage_pv_gen(current_time, pv_diff, available_ev_lst, da_price_lst, im_price_lst):
    
    imbalance_buy = 0
    DISCHRG_THRESH = 0
    
    e_short = np.abs(pv_diff)

    discharge_e_dict = {}
    sum_disch_E = 0
    
    #print("BEFOREEE DISCHRGG E-short ", e_short)
    
    # Obtain the EVs that have POSITIVE LAXITY and ALLOW DISCHARGE
    for ev in available_ev_lst:
        if(ev.laxity > DISCHRG_THRESH and ev.allow_discharge == True and ev.bool_c_d == False):
            # storing the EV obj address in the dict as "key" and the available discharge energy as the "item"
            discharge_e_dict[ev] = get_available_discharge_energy(ev)
    #print('discharge_e_dict ', discharge_e_dict)
    if(len(discharge_e_dict) != 0):
        # Descending sort
        discharge_e_dict = sorted(discharge_e_dict.items(), key=lambda x: x[1], reverse=True)

        for ev in discharge_e_dict:
            sum_disch_E += ev[1]
            
        e_short_orig = e_short
        for ev in discharge_e_dict:
            if (e_short > 0 and ev[1] != 0):
                if(sum_disch_E > e_short):
                    e_discharged = discharge_EV(ev[0], (ev[1]/sum_disch_E)*e_short_orig, e_short)
                    #ev[0].dsch_revenue_contrib += e_discharged * im_price_lst[current_time]
                else:    
                    e_discharged = discharge_EV(ev[0], ev[1], e_short)    
                e_short -= e_discharged
                #print('EEE_DISCCC ', e_discharged)
                #revenue_from_ev_dischrg += e_discharged * im_price_lst[current_time]
        
        #print('SSORRRTT ',e_short)
    return e_short


def get_available_discharge_energy(ev):
    
    e_required = (ev.soc_final - ev.soc_t) * ev.battery_cap
    look_ahead_e_required = ev.alpha_c * ev.eta_c * (ev.stay_time - 1)
    e_discharge = look_ahead_e_required - e_required
    
    if(e_discharge < 0):
        e_discharge = 0
        
    elif(e_discharge >= np.abs(ev.alpha_d)):
        e_discharge = np.abs(ev.alpha_d)
    
    return e_discharge

In [15]:
def discharge_EV(ev, e_discharge_available, e_short):
    
    ETA_D = 0.98
    
    if(e_discharge_available > np.abs(ev.alpha_d)):
        e_discharge_available = np.abs(ev.alpha_d)
    elif(e_discharge_available <= 0):
        e_discharge_available = 0
    
#     if(e_discharge_available != 0):       
#         if(e_short <= e_discharge_available):
#             e_discharge_available = e_short
#             e_short = 0
#         else:
#             e_short -= e_discharge_available

        ev.soc_t = ev.soc_t - (e_discharge_available/ETA_D)/ev.battery_cap
        ev.stay_time -= 1
        ev.laxity = ev.stay_time - ((ev.soc_final - ev.soc_t) * ev.battery_cap)/(ev.alpha_c * ETA_C)
        
        if(ev.laxity == 0):
            ev.laxity = 0
        ev.bool_c_d = True
        
    return e_discharge_available

In [16]:
def update_ev_dict_allowed_discharge_evs(seed, ev_dict, ratio):
        
    rng = np.random.default_rng(seed)
    
    total_evs = 0
    for h in range(24):
        total_evs += len(ev_dict['init_soc']['hour_{}'.format(str(h))])
    
    ev_allowed_discharge = rng.choice(total_evs, size=round(total_evs*(ratio/100)), replace=False)
    
    ev_dict['ev_allowed_discharge'] = ev_allowed_discharge
    return ev_dict

In [17]:
def solve_stage_1_w_expected_vals(seed, date, ratio_EV_discharge):
    
    m = date.month
    
    if(m >= 1 and m <= 3):
        season = 'Winter'
    elif(m >= 4 and m <= 6):
        season = 'Spring'
    elif(m >= 7 and m <= 9):
        season = 'Summer'  
    elif(m >= 10 and m <= 12):
        season = 'Autumn' 
         
    E_PV_gen = util.load_result(r'J:\Thesis_code\thesis_code_saidur\Thesis Code August 2021\new_expected_values\E_PV_{}'.format(season))
    E_im_price = util.load_result(r'J:\Thesis_code\thesis_code_saidur\Thesis Code August 2021\new_expected_values\E_price_imbalance')
    E_da_price = util.load_result(r'J:\Thesis_code\thesis_code_saidur\Thesis Code August 2021\new_expected_values\E_price_da')
    E_EV_dict = util.load_result(r'J:\Thesis_code\thesis_code_saidur\Thesis Code August 2021\new_expected_values\E_EV_dict')
    
    _, bids, _ = latest_deterministic_solver_ev_penetration.main(seed, E_PV_gen, E_da_price, E_im_price, E_EV_dict, ratio_EV_discharge)
    
    return bids

In [18]:
def get_online_alg_result_mc_simul(seed, day_no, current_date, unique_dates, sampling_unique_dates, orig_ev_dict, ratio_EV_discharge, pv_gen_df, price_df, sampling_pv_gen_df, sampling_price_df, num_samples, all_bids_sample_paths):

    # ++++++++++++++++++++++++++++++++++++++++++++++++ STAGE - 1 ++++++++++++++++++++++++++++++++++++++++++
    
    # Load the cached bids using forecasts for stochastic vals.
    
    #all_bids_sample_paths = util.load_result(('hist_mc_sample_paths/{}_perc'.format(ratio_EV_discharge)+'/bid_sample_path_'+str(ratio_EV_discharge)+'_perc'))   
    
    #using 1K
    #all_bids_sample_paths = util.load_result('sample_path_1K/hist_mc_sample_paths/{}_perc'.format(ratio_EV_discharge)+'/bid_sample_path_'+str(ratio_EV_discharge)+'_perc')
    
    # ++++++++++++++++++++++++++++++++++++++++++++++++ STAGE - 2 ++++++++++++++++++++++++++++++++++++++++++
    # Using actual data for online algorithm
    
    DAY_HRS = 24
    
    revenue_sample_paths = []
    im_buy_sample_paths = []
    im_sell_sample_paths = []
    da_revenue_sample_paths = []
    retail_revenue_sample_paths = []

    time_to_soc_sample_paths = []
        
    for b in range(1):
    
        #bids = np.mean(np.array(all_bids_sample_paths)[day_no][0:100], axis=0)
        
        bids = np.mean(all_bids_sample_paths[day_no][0:num_samples], axis=0)

        da_revenue_lst =  np.zeros(DAY_HRS)
        retail_revenue_lst = np.zeros(DAY_HRS)
        imbalance_buy_lst = np.zeros(DAY_HRS)
        imbalance_sell_lst = np.zeros(DAY_HRS)

        revenue_lst = np.zeros(DAY_HRS)

        #_, bids = perform_monte_carlo_simul(seed, date, unique_dates, ratio_EV_discharge, pv_gen_df, price_df)
        ev_dict = update_ev_dict_allowed_discharge_evs(seed, orig_ev_dict, ratio_EV_discharge)

        available_ev_lst = []
        ev_history_lst = []
        avg_time_to_full_soc = []

        pv_gen_lst = np.array(list(pv_gen_df.loc[(pv_gen_df.index == current_date)]['PV_Vol']))
        
        # We do not consider solar gen. so setting it to ZERO
        # ==========================================================================
        #  FIRST CHANGE
        # ==========================================================================
        pv_gen_lst = pv_gen_lst * np.zeros(DAY_HRS)
        
        da_price_lst = np.array(list(price_df.loc[(price_df.index == current_date)]['price_da']))
        im_price_lst = np.array(list(price_df.loc[(price_df.index == current_date)]['price_imbalance']))
    
    
        for idx, p in enumerate(im_price_lst):
            if p < 0:
                im_price_lst[idx] = 0
    
    
        for idx, p in enumerate(da_price_lst):
            if p < 0:
                da_price_lst[idx] = 0
    
        for current_time in range(DAY_HRS):

            revenue = 0
            da_revenue = 0
            im_buy_revenue = 0
            im_sell_revenue = 0
            retail_revenue = 0

            # Get the EVs
            found_evs_lst = get_connected_ev(ev_dict, current_time)

            # Form available_ev_lst
            if(found_evs_lst[0] != -1):
                for found_ev in found_evs_lst:
                    ev_history_lst.append(found_ev)
                    if(ev_history_lst.index(found_ev) in ev_dict['ev_allowed_discharge']):
                        found_ev.allow_discharge = True
                    else:
                        found_ev.allow_discharge = False    
                    available_ev_lst.append(found_ev)

            # Find EVs with Low laxity
            ac_lst = []
            for ev in available_ev_lst:
                ev.bool_c_d = False #Reset for all EVs on every hour
                if(current_time >= ev.departure_time or ev.stay_time == 0 or ev.soc_t >= ev.soc_final):
                    ev.bool_c_d = True
                    ev.completed = True
                elif(ev.soc_t < ev.soc_final): #if(round(ev.soc_t,2) != ev.soc_final):
                    ac_lst.append(charge_EV(ev, ev.alpha_c))

            # Total charging demand
            x_t = np.array(bids[current_time]) + np.sum(np.array(ac_lst))

            bid_amt = np.array(bids[current_time])
            total_chrg_demand =  np.sum(np.array(ac_lst))

    #         im_buy_revenue += (-1 * total_chrg_demand * im_price_lst[current_time])
            da_revenue += bid_amt * da_price_lst[current_time]
            
            # Adding the revenue by getting money from EV owners for charging their batteries
            EV_PWR_RETAIL = 0.13
            
            retail_revenue += total_chrg_demand * EV_PWR_RETAIL
            #da_revenue += total_chrg_demand * EV_PWR_RETAIL

            # Get the surplus/deficit   
#             if(pv_gen_lst[current_time] >= x_t):
#                 # Surplus
#                 pv_diff = pv_gen_lst[current_time] - x_t
#                 im_sell_revenue += surplus_pv_gen(current_time, pv_diff, available_ev_lst, im_price_lst)


#             elif (x_t > pv_gen_lst[current_time]):
#                 # Deficit
#                 pv_diff = pv_gen_lst[current_time] - x_t
#                 e_short = shortage_pv_gen(current_time, pv_diff, available_ev_lst, da_price_lst, im_price_lst)
#                 if(e_short > 0):
#                     im_buy_revenue += -1 * e_short * im_price_lst[current_time]
#                 e_short = 0
                
            # Get the surplus/deficit  
            # NEW for the case W/0 SOLAR
            # ==========================================================================
            #  THIRD CHANGE
            # ==========================================================================
            if (x_t < 0):
                # Surplus
                im_sell_revenue += -1 * x_t * im_price_lst[current_time] #surplus_pv_gen(current_time, pv_diff, available_ev_lst, im_price_lst)
                x_t = 0

            elif (x_t > 0):
                # Deficit
                im_buy_revenue += -1 * x_t * im_price_lst[current_time]
                x_t = 0


            revenue = da_revenue + im_sell_revenue + im_buy_revenue + retail_revenue

            revenue_lst[current_time] = revenue
            da_revenue_lst[current_time] = da_revenue
            imbalance_buy_lst[current_time] = im_buy_revenue
            imbalance_sell_lst[current_time] = im_sell_revenue
            retail_revenue_lst[current_time] = retail_revenue


            for ev in available_ev_lst:                   
                if(ev.bool_c_d == False):
                    # For EVs not charged or discharged, ONLY Laxity update
                    e_required = (ev.soc_final - ev.soc_t) * ev.battery_cap
                    ev.stay_time -= 1 
                    ev.laxity = ev.stay_time - (e_required/(ev.alpha_c * ev.eta_c))    


            for ev in available_ev_lst: 
                if(current_time == ev.departure_time or ev.soc_t >= ev.soc_final or ev.stay_time == 0):
                    if(ev.completed == False):
                        avg_time_to_full_soc.append(1+(current_time - ev.arrival_time))
                    ev.completed = True
                    ev.bool_c_d = True
            
        revenue_sample_paths.append(np.sum(revenue_lst))
        im_buy_sample_paths.append(np.sum(imbalance_buy_lst))
        im_sell_sample_paths.append(np.sum(imbalance_sell_lst))
        da_revenue_sample_paths.append(np.sum(da_revenue_lst))
        retail_revenue_sample_paths.append(np.sum(retail_revenue_lst))
        time_to_soc_sample_paths.append(0)
                        
    return np.average(revenue_sample_paths), np.average(im_buy_sample_paths), np.average(im_sell_sample_paths), np.average(time_to_soc_sample_paths), np.average(da_revenue_sample_paths), np.average(retail_revenue_sample_paths)
    #return np.sum(revenue_lst), np.sum(imbalance_buy_lst), np.sum(imbalance_sell_lst), np.mean(avg_time_to_full_soc)

In [19]:
# def get_online_alg_result_acc_preds(seed, day_no, date, ratio_EV_discharge, pv_gen_lst, da_price_lst, im_price_lst, orig_ev_dict, use_expected_stage_1):

    
#     # ++++++++++++++++++++++++++++++++++++++++++++++++ STAGE - 1 ++++++++++++++++++++++++++++++++++++++++++
        
#     DAY_HRS = 24

#     da_revenue_lst =  np.zeros(DAY_HRS)
#     imbalance_buy_lst = np.zeros(DAY_HRS)
#     imbalance_sell_lst = np.zeros(DAY_HRS)
    
#     revenue_lst = np.zeros(DAY_HRS)    
#  #     all_bids_lst = util.load_result(('hist_acc_bids/{}_perc'.format(ratio_EV_discharge)+'/accurate_bid_'+str(ratio_EV_discharge)+'_perc'))        
# #     bids = all_bids_lst[day_no]
    
# #     ev_dict = update_ev_dict_allowed_discharge_evs(seed, orig_ev_dict, ratio_EV_discharge)

#     if(use_expected_stage_1 == True):
#     # Using Expected values to place bids
#         bids = solve_stage_1_w_expected_vals(seed, date, ratio_EV_discharge)
#         ev_dict = update_ev_dict_allowed_discharge_evs(seed, ev_dict, ratio_EV_discharge)
#     else:
#     # Using Actual values to place bids
#         _, bids, ev_dict = latest_deterministic_solver_ev_penetration.main(seed, pv_gen_lst, da_price_lst, im_price_lst, orig_ev_dict, ratio_EV_discharge)
 
#     # ++++++++++++++++++++++++++++++++++++++++++++++++ STAGE - 2 ++++++++++++++++++++++++++++++++++++++++++
#     # Using actual data for online algorithm
    
#     available_ev_lst = []
#     ev_history_lst = []
#     avg_time_to_full_soc = []
    
#     for current_time in range(DAY_HRS):

#         revenue = 0
#         da_revenue = 0
#         im_buy_revenue = 0
#         im_sell_revenue = 0

#         # Get the EVs
#         found_evs_lst = get_connected_ev(ev_dict, current_time)
                
#         # Form available_ev_lst
#         if(found_evs_lst[0] != -1):
#             for found_ev in found_evs_lst:
#                 ev_history_lst.append(found_ev)
#                 if(ev_history_lst.index(found_ev) in ev_dict['ev_allowed_discharge']):
#                     found_ev.allow_discharge = True
#                 else:
#                     found_ev.allow_discharge = False    
#                 available_ev_lst.append(found_ev)
                             
#         # Find EVs with Low laxity
#         ac_lst = []
#         for ev in available_ev_lst:
#             ev.bool_c_d = False #Reset for all EVs on every hour
#             if(current_time >= ev.departure_time or ev.stay_time == 0 or ev.soc_t >= ev.soc_final):
#                 ev.bool_c_d = True
#                 ev.completed = True
#             elif(ev.soc_t < ev.soc_final): #if(round(ev.soc_t,2) != ev.soc_final):
#                 ac_lst.append(charge_EV(ev, ev.alpha_c))

#         # Total charging demand
#         x_t = np.array(bids[current_time]) + np.sum(np.array(ac_lst))

#         bid_amt = np.array(bids[current_time])
#         total_chrg_demand =  np.sum(np.array(ac_lst))
        
# #         im_buy_revenue += (-1 * total_chrg_demand * im_price_lst[current_time])
#         da_revenue += bid_amt * da_price_lst[current_time]
        
#         # Get the surplus/deficit   
#         if(pv_gen_lst[current_time] >= x_t):
#             # Surplus
#             pv_diff = pv_gen_lst[current_time] - x_t
#             im_sell_revenue += surplus_pv_gen(current_time, pv_diff, available_ev_lst, im_price_lst)
                            
            
#         elif (x_t > pv_gen_lst[current_time]):
#             # Deficit
#             pv_diff = pv_gen_lst[current_time] - x_t
#             e_short = shortage_pv_gen(current_time, pv_diff, available_ev_lst, da_price_lst, im_price_lst)
#             if(e_short > 0):
#                 im_buy_revenue += -1 * e_short * im_price_lst[current_time]
#             e_short = 0
        

#         revenue = da_revenue + im_sell_revenue + im_buy_revenue
         
#         revenue_lst[current_time] = revenue
#         da_revenue_lst[current_time] = da_revenue
#         imbalance_buy_lst[current_time] = im_buy_revenue
#         imbalance_sell_lst[current_time] = im_sell_revenue
        
 #         for ev in available_ev_lst:                   
#             if(ev.bool_c_d == False):
#                 # For EVs not charged or discharged, ONLY Laxity update
#                 e_required = (ev.soc_final - ev.soc_t) * ev.battery_cap
#                 ev.stay_time -= 1 
#                 ev.laxity = ev.stay_time - (e_required/(ev.alpha_c * ev.eta_c))    
    
    
#         for ev in available_ev_lst: 
#             if(current_time == ev.departure_time or ev.soc_t >= ev.soc_final or ev.stay_time == 0):
# #                 print("Done mama wot SoC ", ev.soc_t)
# #                 print('Removed the ev with dept time ', ev.departure_time)
#                 if(ev.completed == False):
#                     avg_time_to_full_soc.append(1+(current_time - ev.arrival_time))
# #                     print("TIME TAKEN TO SOC MAMA ", (1+(current_time - ev.arrival_time)))
#                 ev.completed = True
#                 ev.bool_c_d = True
                        

#     return np.sum(revenue_lst), np.sum(imbalance_buy_lst), np.sum(imbalance_sell_lst), np.mean(avg_time_to_full_soc)

In [20]:
sampling_pv_gen_df = pd.read_csv('sampling_pv_data.csv', index_col='Date')
sampling_pv_gen_df.index = pd.to_datetime(sampling_pv_gen_df.index)

# Javier chages to 2019
pv_gen_test_df = pd.read_csv('real_data/2019_test_data_pv.csv', index_col='Date')
pv_gen_test_df.index = pd.to_datetime(pv_gen_test_df.index)

sampling_price_df = pd.read_csv('sampling_price_data.csv', index_col='Date')
sampling_price_df.index = pd.to_datetime(sampling_price_df.index)

# Javier changes to 2019
price_test_df = pd.read_csv('real_data/2019_test_data_price.csv', index_col='Date')
price_test_df.index = pd.to_datetime(price_test_df.index)

# Javier df_ev
df_ev = pd.read_csv("real_data/df_elaad_preproc.csv", parse_dates = ["starttime_parking", "endtime_parking"])



unique_dates = pv_gen_test_df.index.unique()
sampling_unique_dates = sampling_pv_gen_df.index.unique()

baseline_revenue_dict = {}
online_algo_revenue_dict = {}

seed_lst = [777]
perc_allow_EV_discharge = [0]
#perc_allow_EV_discharge = [0]

for perc in (perc_allow_EV_discharge):

    baseline_revenue_dict = {}
    online_algo_revenue_dict = {}
    
    #all_bids_sample_paths = util.load_result(('hist_mc_sample_paths/{}_perc'.format(perc)+'/bid_sample_path_'+str(perc)+'_perc'))   
    
    all_bids_sample_paths = util.load_result(('bids_no_solar_2019/{}_perc'.format(perc)+'/bid_sample_path_'+str(perc)+'_perc'))
    
    #using 1K    
    #all_bids_sample_paths = np.array(util.load_result('sample_path_1K/hist_mc_sample_paths/{}_perc'.format(perc)+'/bid_sample_path_'+str(perc)+'_perc'))

    
    
    #for num_samples in range(105, 305, 5):
    for _ in range(1):
        
        num_samples = 100

        for run, seed in enumerate(seed_lst):
            baseline_revenue_lst = []

            total_revenue_lst_E_stage_1 = []
            im_buy_revenue_lst_E_stage_1 = []
            im_sell_revenue_lst_E_stage_1 = []
            time_to_full_soc_lst_E_stage_1 = []
            da_revenue_lst_E_stage1 = [] 
            retail_revenue_lst_E_stage1 = []

            total_revenue_lst_R_stage_1 = []
            im_buy_revenue_lst_R_stage_1 = []
            im_sell_revenue_lst_R_stage_1 = []
            time_to_full_soc_lst_R_stage_1 = []
            
            for day_no, date in enumerate(tqdm(unique_dates, total=len(unique_dates))):

                pv_gen_lst = np.array(list(pv_gen_test_df.loc[(pv_gen_test_df.index == date)]['PV_Vol']))
                da_price_lst = np.array(list(price_test_df.loc[(price_test_df.index == date)]['price_da']))
                im_price_lst = (np.array(list(price_test_df.loc[(price_test_df.index == date)]['price_imbalance'])))

                for idx, p in enumerate(im_price_lst):
                    if p < 0:
                        im_price_lst[idx] = 0


                for idx, p in enumerate(da_price_lst):
                    if p < 0:
                        da_price_lst[idx] = 0
                        
                #ev_dict = create_ev_dict(seed+run)
                ev_dict = create_ev_dict_from_df(df_ev, day_no)

                # ONLY IF YOU NEED TO RUN THE OFFLINE ALG
#                 _, bids, _ = latest_deterministic_solver_ev_penetration.main(seed+run, pv_gen_lst, da_price_lst, im_price_lst, ev_dict, perc)
#                 baseline_revenue_lst.append(bids)

                # Using Monte Carlo simulation results
                total, im_buy, im_sell, time_to_full_soc, da_revenue, retail_revenue = get_online_alg_result_mc_simul(seed+run, day_no, date, unique_dates, sampling_unique_dates, ev_dict, perc, pv_gen_test_df, price_test_df, sampling_pv_gen_df, sampling_price_df, num_samples, all_bids_sample_paths)
                total_revenue_lst_E_stage_1.append(total)
                im_buy_revenue_lst_E_stage_1.append(im_buy)
                im_sell_revenue_lst_E_stage_1.append(im_sell)
                time_to_full_soc_lst_E_stage_1.append(time_to_full_soc)
                da_revenue_lst_E_stage1.append(da_revenue)
                retail_revenue_lst_E_stage1.append(retail_revenue)


                # Using accurate predictions
#                 total, im_buy, im_sell, time_to_full_soc = get_online_alg_result_acc_preds(seed+run, day_no,  date, perc, pv_gen_lst, da_price_lst, im_price_lst, ev_dict, False)
#                 total_revenue_lst_R_stage_1.append(total)
#                 im_buy_revenue_lst_R_stage_1.append(im_buy)
#                 im_sell_revenue_lst_R_stage_1.append(im_sell)
#                 time_to_full_soc_lst_R_stage_1.append(time_to_full_soc)

                seed += 10000

            #baseline_revenue_dict[str(perc)] = baseline_revenue_lst

            online_algo_revenue_dict['total_revenue_'+str(perc)+'_samples_'+str(num_samples)+'_E'] = total_revenue_lst_E_stage_1
            online_algo_revenue_dict['im_buy_'+str(perc)+'_samples_'+str(num_samples)+'_E'] = im_buy_revenue_lst_E_stage_1
            online_algo_revenue_dict['im_sell_'+str(perc)+'_samples_'+str(num_samples)+'_E'] = im_sell_revenue_lst_E_stage_1
            online_algo_revenue_dict['time_to_full_soc_'+str(perc)+'_samples_'+str(num_samples)+'_E'] = time_to_full_soc_lst_E_stage_1
            
            online_algo_revenue_dict["da_revenue"+str(perc)+"_samples_"+str(num_samples)+"_E"]  =    da_revenue_lst_E_stage1
            online_algo_revenue_dict["retail_revenue"+str(perc)+"_samples_"+str(num_samples)+"_E"] = retail_revenue_lst_E_stage1


#             online_algo_revenue_dict['total_revenue_'+str(perc)+'_samples_'+str(num_samples)+'_R'] = total_revenue_lst_R_stage_1
#             online_algo_revenue_dict['im_buy_'+str(perc)+'_samples_'+str(num_samples)+'_R'] = im_buy_revenue_lst_R_stage_1
#             online_algo_revenue_dict['im_sell_'+str(perc)+'_samples_'+str(num_samples)+'_R'] = im_sell_revenue_lst_R_stage_1
#             online_algo_revenue_dict['time_to_full_soc_'+str(perc)+'_samples_'+str(num_samples)+'_R'] = time_to_full_soc_lst_R_stage_1

#     util.save_result('w_mc_results/baseline_bids_'+str(perc),baseline_revenue_dict)
#     util.save_result('w_mc_results/mama_1K/online_algo_revenue_ev_asap_'+str(perc),online_algo_revenue_dict)
    util.save_result('w_mc_results/contract_no_solar/ev_asap_contract_revenue_allBidrealEV_'+str(perc),online_algo_revenue_dict)

100%|████████████████████████████████████████████████████████████████| 365/365 [00:06<00:00, 54.25it/s]


In [21]:
vbcvbcvbc

NameError: name 'vbcvbcvbc' is not defined

In [ ]:
price_test_df = pd.read_csv('new_data/new_test_data_price.csv', index_col='Date')
price_test_df.index = pd.to_datetime(price_test_df.index)

In [ ]:
price_test_df['price_imbalance']

In [ ]:
np.percentile(price_test_df['price_imbalance'], 95)

In [ ]:
perc = 0

np.cumsum(util.load_result('w_mc_results/contract_no_solar/ev_asap_contract_revenue_{}'.format(perc))['total_revenue_{}_samples_100_E'.format(perc)])[-1]

In [ ]:
perc = 100

print(np.cumsum(util.load_result('w_mc_results/contract_no_solar/tau_1_contract_online_algo_revenue_original_{}'.format(perc))['total_revenue_{}_run0_E'.format(perc)])[-1])

print(np.cumsum(util.load_result('w_mc_results/contract_no_solar/tau_3_contract_online_algo_revenue_original_{}'.format(perc))['total_revenue_{}_run0_E'.format(perc)])[-1])

#np.cumsum(util.load_result('w_mc_results/contract_no_solar/tau_3_contract_online_algo_revenue_original_{}'.format(perc))['total_revenue_{}_run0_E'.format(perc)])[-1]

In [ ]:
revenue_sample_dict = {}

for s in range(105, 205, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama_1K/online_algo_revenue_ev_asap_0')['total_revenue_0_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))

In [ ]:
fig = plt.gcf()
fig.set_size_inches(20, 16)
fig.use_sticky_edges = True
plt.tight_layout()

revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_0')['total_revenue_0_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))

revenue_sample_dict = {}

for s in range(105, 205, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama_1K/online_algo_revenue_ev_asap_0')['total_revenue_0_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))

In [ ]:
sdfsdfs

In [ ]:
util.load_result('w_mc_results/mama_1K/online_algo_revenue_ev_asap_0').keys()

In [ ]:
util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_0').keys()

In [ ]:
revenue_sample_dict = {}
for s in range(5, 105, 5):
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_0')['total_revenue_0_samples_{}_E'.format(s)])[-1]

In [ ]:
plt.plot(revenue_sample_dict.keys(), revenue_sample_dict.values())

In [ ]:
revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_0')['total_revenue_0_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))

revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_25')['total_revenue_25_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))


revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_50')['total_revenue_50_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))


revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_75')['total_revenue_75_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))


revenue_sample_dict = {}

for s in range(5, 105, 5):    
    revenue_sample_dict[str(s)] = np.cumsum(util.load_result('w_mc_results/mama/online_algo_revenue_ev_asap_100')['total_revenue_100_samples_{}_E'.format(s)])[-1]
plt.plot(revenue_sample_dict.keys(), [0] + list(np.diff(list(revenue_sample_dict.values()))))

In [ ]:
hjgfckjfvkhj

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

ev_asap_E_revenue_lst = []
ev_asap_R_revenue_lst = []

for p in [0]:

    total = np.array(util.load_result('w_mc_results/mama/neg_sample_online_algo_revenue_ev_asap_{}'.format(p))['total_revenue_{}_run0_E'.format(p)])
    ev_asap_E_revenue_lst.append(np.cumsum(total)[-1])
    total = np.array(util.load_result('w_mc_results/neg_sample_online_algo_revenue_ev_asap_{}'.format(p))['total_revenue_{}_run0_R'.format(p)])
    ev_asap_R_revenue_lst.append(np.cumsum(total)[-1])

In [ ]:
util.load_result('from_snowball/june_22/safe_rl/mc_sample_SAC_RL_result_test_0')['revenue_reward']

In [ ]:
util.load_result('from_snowball/june_22/safe_rl/mc_sample_SAC_RL_result_train_100')['mean_rewards']

In [ ]:
np.cumsum(util.load_result('from_snowball/june_22/safe_rl/mc_sample_SAC_RL_result_test_100')['revenue_reward'][1][0])[-1]

In [ ]:
ev_asap_R_revenue_lst

In [ ]:
ev_asap_E_revenue_lst

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

ev_asap_R_revenue_lst = []

for p in [0]:

    total = np.array(util.load_result('online_alg_results/online_algo_revenue_ev_asap_{}'.format(p))['total_revenue_{}_run0_R'.format(p)])
    ev_asap_R_revenue_lst.append(np.cumsum(total)[-1])

In [ ]:
ev_asap_R_revenue_lst

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['total_revenue_0_run0_E'])
im_buy = np.array(util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['im_buy_0_run0_E'])
im_sell = util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['im_sell_0_run0_E']
np.cumsum(total)[-1]

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['total_revenue_0_run0_R'])
im_buy = np.array(util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['im_buy_0_run0_R'])
im_sell = util.load_result('w_mc_results/online_algo_revenue_ev_asap_0')['im_sell_0_run0_R']
np.cumsum(total)[-1]

In [ ]:
asdasdaasdasd

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['total_revenue_0_run0_E'])
im_buy = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['im_buy_0_run0_E'])
im_sell = util.load_result('w_mc_results/online_algo_revenue_original_0')['im_sell_0_run0_E']
np.cumsum(total)[-1]

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['total_revenue_0_run0_E'])
im_buy = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['im_buy_0_run0_E'])
im_sell = util.load_result('w_mc_results/online_algo_revenue_original_0')['im_sell_0_run0_E']
np.cumsum(total)[-1]

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['total_revenue_0_run0_R'])
im_buy = np.array(util.load_result('w_mc_results/online_algo_revenue_original_0')['im_buy_0_run0_R'])
im_sell = util.load_result('w_mc_results/online_algo_revenue_original_0')['im_sell_0_run0_R']
np.cumsum(total)[-1]

In [ ]:
#util.load_result('w_mc_results/online_algo_revenue_original_50')

total = np.array(util.load_result('online_alg_results/online_algo_revenue_original_0')['total_revenue_0_run0_R'])
im_buy = np.array(util.load_result('online_alg_results/online_algo_revenue_original_0')['im_buy_0_run0_R'])
im_sell = util.load_result('online_alg_results/online_algo_revenue_original_0')['im_sell_0_run0_R']
np.cumsum(total)[-1]

In [ ]:
np.cumsum(im_buy)[-1]

In [ ]:
np.cumsum(im_sell)[-1]

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_0')['total_revenue_0_run0_E'])[-1])
print('Using Actual ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_0')['total_revenue_0_run0_R'])[-1])

In [ ]:
print('Using Expec 0',np.cumsum(util.load_result('online_alg_results/baseline_revenue_0')['0'])[-1])
print('Using Expec 0',np.cumsum(util.load_result('online_alg_results/baseline_revenue_100')['100'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_ev_asap_0')['total_revenue_0_run0_E'])[-1])
print('Using Actual ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_ev_asap_0')['total_revenue_0_run0_R'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_25')['total_revenue_25_run0_E'])[-1])
print('Using Actual ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_25')['total_revenue_25_run0_R'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_100')['total_revenue_100_run2_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_0')['total_revenue_0_run0_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_original_75')['total_revenue_75_run1_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/online_algo_revenue_randomized_75')['total_revenue_75_run1_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/randomized/50_50/online_algo_revenue_randomized_75_50_50')['total_revenue_75_run1_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/randomized/90_10/online_algo_revenue_randomized_75_90_10')['total_revenue_75_run1_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/randomized/omid/90_10/online_algo_revenue_randomized_omid_75_90_10')['total_revenue_75_run1_E'])[-1])

In [ ]:
print('Using Expec ',np.cumsum(util.load_result('online_alg_results/randomized/omid/50_50/online_algo_revenue_randomized_omid_75_50_50')['total_revenue_75_run1_E'])[-1])

In [ ]:
ratio_EV_discharge = 0

for day_num, date

np.mean(np.array(util.load_result(('sample_path_1K/hist_mc_sample_paths/{}_perc'.format(ratio_EV_discharge)+'/bid_sample_path_'+str(ratio_EV_discharge)+'_perc')))[day_num][0:NUM_SAMPLES],axis=0)

In [ ]:
np.mean(np.array(util.load_result(('sample_path_1K/hist_mc_sample_paths/{}_perc'.format(ratio_EV_discharge)+'/bid_sample_path_'+str(ratio_EV_discharge)+'_perc')))[0][0:NUM_SAMPLES], axis=0)

In [ ]:
for idx, val in enumerate([1]):
    print(idx)
    print(val)